# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [59]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import gmaps.datasets
from pprint import pprint
# Import API key
from config import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [11]:
citiesPath = "output_data/cities.csv"
cw_df= pd.read_csv(citiesPath) 
cw_df.head(3)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Cape Town,-33.9258,18.4232,66.99,88,0,3.44,ZA,1613192304
1,Vanavara,60.3400,102.2797,-10.95,87,100,3.65,RU,1613192304
2,Marrakesh,31.6315,-8.0083,51.80,82,0,3.44,MA,1613192304


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [136]:
gmaps.configure(g_key)
data=[(-33.9258, 18.4232)]
figure_layout={
    'width': '700px',
    'height': '600px',
    'border':'1px solid black',
    'padding':'1px',
    'margin':'0 auto 0 auto'
}
fig=gmaps.figure(layout=figure_layout, map_type= 'SATELLITE')

In [137]:
coord=cw_df[['Lat', 'Lng']]
weights=cw_df[ 'Humidity']
fig.add_layer(gmaps.heatmap_layer(coord, weights=weights))
heatmap_layer.max_intensity = 100
heatmap_layer = gmaps.heatmap_layer(coord, point_radius=10)

fig


Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [75]:
hotel_df=cw_df.loc[(cw_df['Max Temp']<85)&(cw_df['Max Temp']>77)&
                         (cw_df['Wind Speed']<10)&
                          (cw_df['Cloudiness']<10),:].reset_index(drop=True)
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Busselton,-33.6500,115.3333,81.97,37,0,5.79,AU,1613192313
1,Chalisgaon,20.4667,75.0167,83.61,17,0,3.56,IN,1613192314
2,Avarua,-21.2078,-159.7750,84.20,65,1,4.61,CK,1613192314
3,Gopālpur,19.2667,84.9167,80.82,41,0,6.31,IN,1613192317
4,Harper,4.3750,-7.7169,78.98,89,0,6.64,LR,1613192352
5,Acaraú,-2.8856,-40.1200,78.21,87,1,9.78,BR,1613192355
6,Tura,25.5198,90.2201,80.11,31,0,3.04,IN,1613192356
7,Poum,-20.2333,164.0167,81.25,70,1,8.75,NC,1613192361
8,Deniliquin,-35.5333,144.9667,80.01,36,0,5.99,AU,1613192365
9,Grand Gaube,-20.0064,57.6608,82.99,69,7,8.99,MU,1613192493


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [111]:
hotelname= []

In [113]:
for x in range(len(hotel_df)):
    location= str(hotel_df['Lat'][x])+','+str(hotel_df['Lng'][x])
    url=f'https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={location}&radius=5000&type=lodging&key={g_key}'
    data= requests.get(url).json() 
    hotelname.append(data['results'][1]['name'])

In [125]:
hotel_df=hotel_df
hotel_df['Hotel Name']=hotelname
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Busselton,-33.6500,115.3333,81.97,37,0,5.79,AU,1613192313,Restawile Motel
1,Chalisgaon,20.4667,75.0167,83.61,17,0,3.56,IN,1613192314,"Chaudhari Mangal Karyalaya, Chalisgaon"
2,Avarua,-21.2078,-159.7750,84.20,65,1,4.61,CK,1613192314,Club Raro Resort
3,Gopālpur,19.2667,84.9167,80.82,41,0,6.31,IN,1613192317,"MAYFAIR Palm Beach Resort, Gopalpur"
4,Harper,4.3750,-7.7169,78.98,89,0,6.64,LR,1613192352,Cape View Guest House
5,Acaraú,-2.8856,-40.1200,78.21,87,1,9.78,BR,1613192355,Castelo Encantado
6,Tura,25.5198,90.2201,80.11,31,0,3.04,IN,1613192356,Hotel Sundare
7,Poum,-20.2333,164.0167,81.25,70,1,8.75,NC,1613192361,Guégué Eden
8,Deniliquin,-35.5333,144.9667,80.01,36,0,5.99,AU,1613192365,McLean Beach Holiday Park
9,Grand Gaube,-20.0064,57.6608,82.99,69,7,8.99,MU,1613192493,Zilwa Attitude Hotel


In [132]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [138]:
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…